# Problem Solving Steps
1. Write out problem statement
2. Ask clarifying questions pertaining to problem
3. Re-state your understanding of the problem, and confirm your understanding of the problem

## FizzBuzz

In [1]:
def fizzbuzz(x):
    if (x % 3 == 0) & (x % 5 == 0):
        return "FizzBuzz"
    elif x % 3 == 0:
        return "Fizz"
    elif x % 5 == 0:
        return "Buzz"
    else:
        return x

In [2]:
for i in range(1, 101):
    print(fizzbuzz(i))

In [3]:
# classifying numbers as fizzbuzz is the goal; 

## Keras Workflow
1. Setup training data
2. Setup labels
3. Encode inputs
4. Write a decoder for the outputs
5. Build a model, fit/train on training data
6. Evaluate the model on test data
7. We can use .predict_classes on out-of-sample data to make predictions

In [4]:
import warnings
warnings.filterwarnings('ignore')

# imports
import numpy as np
np.random.seed(23)
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model

Using TensorFlow backend.


In [5]:
# specify the number of binary digits, variable so we can change it
num_digits = 12

In [6]:
# let's setup out training data
# raw_training_data = np.arange(101, 2**num_digits)
raw_training_data = np.random.randint(101, 2**num_digits, 2**num_digits - 102)

In [7]:
raw_training_data[:10]

array([ 696,  843, 1165, 2094, 3099, 1613, 2180, 2386, 3633,  448])

In [8]:
# one hot encode fizzbuzz
def encode_fizzbuzz(x):
    """OHE fizzbuzz:
    
    [0, 0, 0, 1] == FizzBuzz
    [0, 0, 1, 0] == Fizz
    [0, 1, 0, 0] == Buzz
    [1, 0, 0, 0] == number not FizzBuzz, Fizz, or Buzz"""
    
    if (x % 3 == 0) & (x % 5 == 0):
        return np.array([0, 0, 0, 1]) # encoding for "FizzBuzz"
    elif x % 5 == 0: 
        return np.array([0, 0, 1, 0]) # encoding for "Buzz"
    elif x % 3  == 0: 
        return np.array([0, 1, 0, 0]) # encoding for "Fizz"
    else:
        return np.array([1, 0, 0, 0]) # encoding for the number output

In [9]:
# needs two inputs b/c we have the one hot encoding but we also need the number itself
# def decode_fizzbuzz(one_hot_fizzbuzz, i):
#     return ["FizzBuzz", "Fizz", "Buzz", str(i)][one_hot_fizzbuzz]

In [10]:
# binary encode function
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

In [11]:
# convert from binary to fizzbuzz output
def decode_fizzbuzz(i, prediction):
    return [str(i), "Fizz", "Buzz", "FizzBuzz"][prediction]

In [12]:
encoded_training_data = np.array([binary_encode(i, num_digits) for i in raw_training_data])

In [13]:
x_train = encoded_training_data
y_train = np.array([encode_fizzbuzz(i) for i in raw_training_data])

In [14]:
print(raw_training_data[0])
print(x_train[0])
print(y_train[0])

696
[0 0 0 1 1 1 0 1 0 1 0 0]
[0 1 0 0]


In [15]:
model = Sequential()

In [16]:
model.add(Dense(1000, input_dim=num_digits, activation="relu"))
model.add(Dense(1000, activation="relu"))

model.add(Dense(4, activation="softmax"))

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              13000     
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 4004      
Total params: 1,018,004
Trainable params: 1,018,004
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss="categorical_crossentropy", optimizer="adagrad", metrics=["accuracy"])

In [19]:
model.fit(x_train, y_train, nb_epoch=30, batch_size=128)

Epoch 1/30
3994/3994 [==============================] - 1s 257us/step - loss: 1.3320 - accuracy: 0.4977
Epoch 2/30
3994/3994 [==============================] - 0s 120us/step - loss: 1.1436 - accuracy: 0.5260
Epoch 3/30
3994/3994 [==============================] - 0s 121us/step - loss: 1.1334 - accuracy: 0.5280
Epoch 4/30
3994/3994 [==============================] - 0s 120us/step - loss: 1.1119 - accuracy: 0.5320
Epoch 5/30
3994/3994 [==============================] - 0s 118us/step - loss: 1.0903 - accuracy: 0.5371
Epoch 6/30
3994/3994 [==============================] - 0s 121us/step - loss: 1.0588 - accuracy: 0.5461
Epoch 7/30
3994/3994 [==============================] - 0s 122us/step - loss: 1.0203 - accuracy: 0.5566
Epoch 8/30
3994/3994 [==============================] - 0s 120us/step - loss: 0.9738 - accuracy: 0.5794
Epoch 9/30
3994/3994 [==============================] - 0s 122us/step - loss: 0.9163 - accuracy: 0.6079
Epoch 10/30
3994/3994 [==============================] - 0s 121u

In [20]:
numbers = np.arange(1, 101)
x_test = np.transpose(binary_encode(numbers, num_digits))
x_test[0:2]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [21]:
y_test = model.predict_classes(x_test)

In [22]:
predictions = np.vectorize(decode_fizzbuzz)(numbers, y_test)

In [23]:
predictions

array(['1', '2', 'Fizz', '4', 'Buzz', 'Fizz', '7', '8', 'Fizz', 'Buzz',
       '11', 'Fizz', '13', '14', 'FizzBuzz', '16', '17', '18', '19', '20',
       'Fizz', '22', '23', 'Fizz', 'Buzz', '26', 'Fizz', '28', '29',
       'FizzBuzz', '31', '32', '33', '34', 'Buzz', 'Fizz', '37', '38',
       'Fizz', 'Buzz', '41', 'Fizz', '43', '44', 'FizzBuzz', 'Fizz', '47',
       '48', '49', '50', 'Fizz', '52', '53', 'Fizz', 'Buzz', '56', 'Fizz',
       '58', '59', 'FizzBuzz', '61', '62', 'Fizz', '64', 'Buzz', 'Fizz',
       '67', '68', 'Fizz', 'Buzz', '71', 'Fizz', '73', '74', 'FizzBuzz',
       'Fizz', '77', 'Fizz', '79', '80', 'Fizz', '82', '83', 'Fizz',
       'Buzz', '86', 'Fizz', '88', '89', 'FizzBuzz', '91', '92', 'Fizz',
       '94', 'Buzz', 'Fizz', '97', '98', 'Fizz', 'Buzz'], dtype='<U8')

In [24]:
actual = np.array([fizzbuzz(x) for x in range(1, 101)])
actual

array(['1', '2', 'Fizz', '4', 'Buzz', 'Fizz', '7', '8', 'Fizz', 'Buzz',
       '11', 'Fizz', '13', '14', 'FizzBuzz', '16', '17', 'Fizz', '19',
       'Buzz', 'Fizz', '22', '23', 'Fizz', 'Buzz', '26', 'Fizz', '28',
       '29', 'FizzBuzz', '31', '32', 'Fizz', '34', 'Buzz', 'Fizz', '37',
       '38', 'Fizz', 'Buzz', '41', 'Fizz', '43', '44', 'FizzBuzz', '46',
       '47', 'Fizz', '49', 'Buzz', 'Fizz', '52', '53', 'Fizz', 'Buzz',
       '56', 'Fizz', '58', '59', 'FizzBuzz', '61', '62', 'Fizz', '64',
       'Buzz', 'Fizz', '67', '68', 'Fizz', 'Buzz', '71', 'Fizz', '73',
       '74', 'FizzBuzz', '76', '77', 'Fizz', '79', 'Buzz', 'Fizz', '82',
       '83', 'Fizz', 'Buzz', '86', 'Fizz', '88', '89', 'FizzBuzz', '91',
       '92', 'Fizz', '94', 'Buzz', 'Fizz', '97', '98', 'Fizz', 'Buzz'],
      dtype='<U21')

In [25]:
(actual == predictions).mean()

0.92